In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

### 재료입력

In [11]:
#### 재료입력
have_list = input("재료를 입력하세요: (단 재료들은 spacebar로 구분하세요)")

재료를 입력하세요: (단 재료들은 spacebar로 구분하세요)가지 소고기


# Recipe List

In [12]:
ingredients = []
def get_recipe():
    value = have_list.split(',')

    q = ''
    for val in value:
        q = q + val + ' '
    q = q[:-1]
    
    base_url = 'http://www.10000recipe.com/recipe/list.html'

    title_list = []
    url_list = []

    for i in range(1):
        params = {
            'q': q,
            'page': i+1 
        }
        resp = requests.get(base_url, params=params)
        soup = BeautifulSoup(resp.text)
        ul_tag = soup.find_all('ul', class_='common_sp_list_ul ea4')

        for ul in ul_tag:        
            div_tags = ul.select('li div.common_sp_caption_tit.line2')  # title 추출 위함
            a_tags = ul.select('li>div>a')  # url 추출 위함

            for div, a in zip(div_tags, a_tags):
                title_list.append(div.text)
                url_list.append(a.get('href'))

    #### base_url + level_list
    import urllib
    urllib.parse.urljoin(base_url, url_list[0])
    url_list = list(map(lambda url: urllib.parse.urljoin(base_url, url), url_list))


    #### Time_list & Ingredient_list
    time_list = []
    level_list = []

    for url in url_list:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)

        div_tag = soup.find('div', class_='view2_summary_info')
        
 
            
        
        time_tag = div_tag.select_one('span.view2_summary_info2')
        if time_tag:
            time_text = time_tag.text
        else:
            time_text = None

        level_tag = div_tag.select_one('span.view2_summary_info3')

        if level_tag:
            level_text = level_tag.text
        else:
            level_text = None

        level_list.append(level_text)
        time_list.append(time_text)

    #### Ingredient_list
    ingredient_list = []
    for url in url_list:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)

        div_tag = soup.find('div', class_='ready_ingre3')  

        if not div_tag:
            ingredient_list.append([None, None])
            continue

        temp_list = []
        li_tag = div_tag.select('ul > a > li')
        for li in li_tag:
            name = li.text.strip('\n')
            source = name.split('\n')
            if len(source) == 1:
                source = [source[0].strip(), None]
            else:
                source = [source[0].strip(), source[1].strip()]

            temp_list.append(source)
        ingredient_list.append(temp_list)
    
                    
    ## 재료만 따로 저장
    for i in ingredient_list:
        temp = []
        for j in i:
            food = j
            temp.append(food)
        ingredients.append(temp)
        
    ## DataFrame
    df = pd.DataFrame(title_list)
    df.columns = ['title']
    df['url'] = url_list
    df['time'] = time_list
    df['level'] = level_list
    df['ingredient'] = ingredient_list
    return df

In [13]:
get_recipe()

,title,url,time,level,ingredient
0,소고기 배춧국 만들기 든든한 한끼로 제격!,http://www.10000recipe.com/recipe/6880695,30분 이내,아무나,"[[배추잎, 5~6장], [소고기국거리, 200g], [다진마늘, 1/2스푼], [..."
1,가지 맛있게 먹는 방법! 초대박 가지요리 레시피 가지탕수,http://www.10000recipe.com/recipe/6896720,60분 이내,중급,"[[가지, 1개], [튀김가루, None], [다진양파, 1/4개], [다진마늘, ..."
2,소고기 대파 볶음,http://www.10000recipe.com/recipe/6899457,15분 이내,초급,"[[한우 소고기 (살치살), 200g], [대파(흰부분), 1컵], [식용유, 2큰..."
3,가지 그라탱 / eggplant Gratain,http://www.10000recipe.com/recipe/6906406,30분 이내,아무나,"[[가지, 2개], [양파, 1개], [토마토소스, 7Ts], [모짜렐라 치즈, 2..."
4,[No.3 양배추 소고기 덮밥] 겨울의 외로움으로부터 힐링시켜주는 따듯한 덮밥 / ...,http://www.10000recipe.com/recipe/6841467,30분 이내,초급,"[[소고기, None], [양배추, None], [쌀, None], [전분가루, N..."
5,소고기 감자조림 일본가정식,http://www.10000recipe.com/recipe/6828936,None,초급,"[[소고기 불고기용, 300g], [감자, 4개], [청고추, 2개], [홍고추, ..."
6,수미네 반찬 소고기 뭇국 끓이기. 경상도식이 더 맛있네~ 간단한 국 요리,http://www.10000recipe.com/recipe/6934526,60분 이내,아무나,"[[소고기 국거리용, 1주먹], [무, 1/2개], [대파, 1대], [다진마늘, ..."
7,중화풍 가지 덮밥 제철요리,http://www.10000recipe.com/recipe/6873624,15분 이내,아무나,"[[밥, 1공기], [가지, 1개], [양파, 1/2개], [대파, 1토막], [매..."
8,진정한 밥도둑 소고기 볶음 고추장,http://www.10000recipe.com/recipe/6838094,15분 이내,초급,"[[갈은 소고기, 480g], [양파, 2개], [고추장, 9T], [다진 마늘, ..."
9,소고기 파프리카말이,http://www.10000recipe.com/recipe/6796725,30분 이내,초급,"[[소고기 불고기감, 150g], [파프리카, 1개], [깻잎, 10장], [간장,..."


In [14]:
def no_matter(num):
    # 필요한 재료
    want_ingredient = ingredients[num]
    
    input_ingredient = have_list.split(' ')
    
    # 장바구니 넣어야 할 항목
    for have in input_ingredient:
        if have in want_ingredient:
            want_ingredient.remove(have)
    
    return want_ingredient

In [15]:
no_matter(5)

[['소고기 불고기용', '300g'],
 ['감자', '4개'],
 ['청고추', '2개'],
 ['홍고추', '2개'],
 ['물', '1머그컵'],
 ['간장', '3큰술'],
 ['올리고당', '2큰술'],
 ['참기름', '1큰술'],
 ['통깨', '1큰술'],
 ['물', '1머그컵'],
 ['간장', '3큰술'],
 ['올리고당', '2큰술'],
 ['참기름', '1큰술'],
 ['통깨', '1큰술'],
 ['청주', '2큰술'],
 ['후추', '약간']]

---

# 마켓컬리

In [142]:
def market(ingr):
    ## Selenium
    # 로그인
    driver = webdriver.Chrome('crawling/chromedriver.exe')
    driver.get('https://www.kurly.com/shop/member/login.php') 
    id_input = driver.find_element_by_name('m_id')
    id_input.send_keys('yoo100567')
    pw_input = driver.find_element_by_name('password')
    pw_input.send_keys('kp0k270324!')
    login = driver.find_element_by_class_name('btn_type1')
    login.click()

    # 검색창 비우기
    sword = driver.find_element_by_id('sword')
    sword.clear()

    # 장바구니 넣기
    for want in ingr:
        sword = driver.find_element_by_name('sword')  # 검색창
        sword.clear()  # 검색창에 쓰여진것 지우기
        sword.send_keys(want)  # 재료 리스트 입력
        btn_search = driver.find_elements_by_class_name('btn_search')[0]
        btn_search.click()  # '검색버튼' 클릭

        # 장바구니 클릭
        driver.implicitly_wait(10000)
        xpath = driver.find_element_by_xpath('//*[@id="goodsList"]/div[1]/div/ul/li[1]/div/div/div/button')
        xpath.click()
        #xpath = driver.find_elements_by_css_selector('.btn.btn_cart')[0]
        #xpath.click()

        # 장바구니 큰 틀
        inner_option = driver.find_elements_by_class_name('inner_option')[0]

        # 장바구니 추가 버튼
        final = inner_option.find_element_by_class_name('btn_type1')


        # 옵션 이름 리스트
        name = inner_option.find_elements_by_class_name('name')   
        name_list = []
        for i in name:
            name_list.append(i.text)


        for idx, item in enumerate(name_list):
            if len(name_list) == 1:  # name_list의 길이가 1인 경우

                if '품절' not in item:
                    final.click()
                else:
                    btn_close = driver.find_element_by_class_name('btn_close1')[0]  # 장바구니 닫기
                    btn_close.click()
                    break  # 첫번째 for문으로 돌아가야 하는데 맞나?????????????????


            else: # name_list의 길이가 2 이상인 경우
                if '품절' not in item:

                    if want in item:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[idx]  # 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        ## 첫번째 for문으로 다시 돌아가야 하는데 맞나????????

                    else:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[0]  # 첫번째 항목 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 P담기

                    break

                else:  # 품절이 항목에 있는 경우

                    if '품절' in item:
                        pass
                    elif want in item:
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[idx]  # 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        break
                    else:    
                        button_up = inner_option.find_elements_by_css_selector('.btn.up')[0]  # 첫번째 항목 수량 올리기
                        button_up.click()
                        final.click()  # 장바구니 담기
                        break